In [6]:
import chess
import chess.pgn
import chess.engine
import re

def convertion_en_board(fen):
    return chess.Board(fen)
########################################################################################################
#Si la matériel est différent d'un coup à l'autre (i.e l'adversaire vient de prendre une pièce) mais qu'il est possible de pouvoir recapturer une pièce il faut que le score 
#renvoyé par eval_diff_mat n'est pas d'importance. Il faut donc pouvoir calculer la différence de matériel entre deux coup.
########################################################################################################
# les features : 

PIECE_VALUES = {
    chess.PAWN:   (100, 100),
    chess.KNIGHT: (320, 310),
    chess.BISHOP: (330, 320),
    chess.ROOK:   (500, 510),
    chess.QUEEN:  (900, 920),
    chess.KING:   (0, 0)
}


def eval_mat(board: chess.Board, state_game):
    score_white_mg = 0
    score_black_mg = 0
    score_white_eg = 0
    score_black_eg = 0

    for piece_type in PIECE_VALUES:
        mg_value, eg_value = PIECE_VALUES[piece_type]

        white_pieces = board.pieces(piece_type, chess.WHITE)
        black_pieces = board.pieces(piece_type, chess.BLACK)

        score_white_mg += mg_value * len(white_pieces)
        score_black_mg += mg_value * len(black_pieces)

        score_white_eg += eg_value * len(white_pieces)
        score_black_eg += eg_value * len(black_pieces)

    if state_game == "mg":
        return score_white_mg, score_black_mg
    return score_white_eg, score_black_eg

       
def is_attacking_more_valuable_piece(board, square):
    piece = board.piece_at(square)
    if not piece:
        return False  # Pas de pièce à cette position

    piece_value = PIECE_VALUES[piece.piece_type]
    attacks = board.attacks(square)

    for target_square in attacks:
        target_piece = board.piece_at(target_square)
        if target_piece and target_piece.color != piece.color:
            target_value = PIECE_VALUES[target_piece.piece_type]
            if target_value >= piece_value:
                return True

    return False




def eval_diff_mat(board, state):
    a,b = eval_mat(board,state)
    
    return a-b #par convention si les blancs ont plus de matériel, eval_diff_mat > 0

def eval_finale_pièces(board_n, state, color):
    reel_diff = 0
    color_oppose = not color
    valeur = 0
    if abs(eval_diff_mat(board_n, state))>=230: #il y'a une différence significative de matériel 
        for square in board_n.piece_map():
            attaquants_de_notre_couleur = 0
            defenseur_de_couleur_oppose = 0
            piece = board_n.piece_at(square)
            if state == "eg":
                valeur = PIECE_VALUES[piece.piece_type][1]
            valeur = PIECE_VALUES[piece.piece_type][0]
            if piece.color == color_oppose and valeur > 300:#pièce de haute valeur
                for k in board.attackers(color, square):
                    piece = board.piece_at(k)
                    if piece:
                        attaquants_de_notre_couleur += 1
                for k in board.attackers(not color, square):
                    piece = board.piece_at(k)
                    if piece:
                        defenseur_de_couleur_oppose += 1
            if attaquants_de_notre_couleur > defenseur_de_couleur_oppose: #alors la pièce est en prise et l'avantage matériel n'est pas réellement
                reel_diff+= valeur
        if color == chess.WHITE:
            return eval_diff_mat(board_n, state) + reel_diff
        return - eval_diff_mat(board_n, state) + reel_diff
    return eval_diff_mat(board_n, state)



                


            



def square_color(square):
    return (chess.square_file(square) + chess.square_rank(square)) % 2 == 0


#############################################################################################################
##1LES CAVALIERS
#Fonctions utilitaires de caractérisation de la bonne position d'un cavalier

def is_outpost(board, square, color):
    rank = chess.square_rank(square)
    # Détermine si cavalier est avancé et ne peut pas être chassé facilement
    if color == chess.WHITE and rank < 4:
        return False
    if color == chess.BLACK and rank > 5:
        return False

    # Vérifie s’il est soutenu par un pion
    if not is_supported_by_pawn(board, square, color):
        return False

    # Vérifie si un pion adverse peut le chasser
    if is_attacked_by_pawn(square, board, color):
        return False

    return True

def is_supported_by_pawn(board, square, color):
    rank = chess.square_rank(square)
    file = chess.square_file(square)
    direction = -1 if color == chess.WHITE else 1
    for offset in [-1, 1]:
        adj_file = file + offset
        adj_rank = rank + direction
        if 0 <= adj_file <= 7 and 0 <= adj_rank <= 7:
            adj_sq = chess.square(adj_file, adj_rank)
            piece = board.piece_at(adj_sq)
            if piece and piece.piece_type == chess.PAWN and piece.color == color:
                return True
    return False
def is_attacked_by_pawn(square, board, color):
    # Détermine la direction d'attaque des pions adverses
    direction = 1 if color == chess.BLACK else -1
    file = chess.square_file(square)
    rank = chess.square_rank(square)
    for offset in [-1, 1]:  # gauche et droite
        target_file = file + offset
        target_rank = rank + direction
        if 0 <= target_file <= 7 and 0 <= target_rank <= 7:
            from_square = chess.square(target_file, target_rank)
            piece = board.piece_at(from_square)
            if piece and piece.color == color and piece.piece_type == chess.PAWN:
                return True

    return False
#Fonction qui évalue :
def evaluate_knights(board, color):
    score = 0
    knight_squares = board.pieces(chess.KNIGHT, color)

    for square in knight_squares:
        rank = chess.square_rank(square)
        file = chess.square_file(square)

        # Critère 1 : avant-poste
        if is_outpost(board, square, color):
            score += 40  # Bonus pour avant-poste

        # Critère 2 : attaques importantes
        attacked = board.attacks(square)
        for target_square in attacked:
            target_piece = board.piece_at(target_square)
            if target_piece and target_piece.color != color:  # Vérifie que c’est une pièce adverse
                if target_piece.piece_type == chess.ROOK:
                    score += 25
                elif target_piece.piece_type == chess.QUEEN:
                    score += 40
                elif not board.is_attacked_by(color, target_square):  # pièce non protégée
                    score += 15

        # Critère 3 : mobilité
        mobility = sum(
            1 for sq in board.attacks(square)
            if not board.is_attacked_by(not color, sq)
        )
        score += mobility * 3
        # Critère 4 : centralisation
        if square in [chess.D4, chess.D5, chess.E4, chess.E5]:
            score += 30
        # Critère 5 : pour les cases d'avant-poste potentielles que le cavalier pourrait atteindre
        for piece_sq in attacked:
            attackers = 0
            defenders = 0
            for k in board.attackers(color, piece_sq):
                if k != square:
                    piece = board.piece_at(k)
                    if piece and piece.piece_type != chess.PAWN:
                        defenders += 1
            for k in board.attackers(not color, piece_sq):
                piece = board.piece_at(k)
                if piece.piece_type != chess.PAWN:
                    attackers += 1
            if defenders > attackers and not is_attacked_by_pawn(piece_sq, board, not color):
                score += 4

    return score






########################################################################################################
##2 La sécurité du roi 
def evaluate_king_safety(board, color): #Si la valeur est elevé, le roi est en grand danger
    score = 0
    king_square = board.king(color)

    if king_square is None:
        return 1000  # Cas extrême : roi absent (erreur ou mat)

    rank = chess.square_rank(king_square)
    file = chess.square_file(king_square)

    # 1. Roi au centre vs roqué
    if file in [3, 4]:  # Cases centrales (d/e)
        score += 10
    elif file in [2, 5]:  # Plus excentré, sans être trop sur les bords
        score += 5 

    # 2. Structure de pions autour du roi (bouclier)
    pawn_shield = [
        (file - 1, rank + (1 if color == chess.WHITE else -1)),
        (file,     rank + (1 if color == chess.WHITE else -1)),
        (file + 1, rank + (1 if color == chess.WHITE else -1)),
    ]
    for f, r in pawn_shield:
        if 0 <= f <= 7 and 0 <= r <= 7:
            sq = chess.square(f, r)
            piece = board.piece_at(sq)
            if piece and piece.piece_type == chess.PAWN and piece.color == color:
                score = score
            else:
                score += 14

    # 3. Colonne ouverte / semi-ouverte sur le roi
    column_has_enemy_rooks_or_queen = False
    non_king_piece_count = 0
    for r in range(8):
        sq = chess.square(file, r)
        piece = board.piece_at(sq)
        if piece:
            if piece.color != color and piece.piece_type in [chess.ROOK, chess.QUEEN]:
                column_has_enemy_rooks_or_queen = True
            elif piece.piece_type in [chess.PAWN, chess.KNIGHT, chess.BISHOP]:
                non_king_piece_count += 1
    if column_has_enemy_rooks_or_queen and non_king_piece_count <= 1 :
        score += 15

    # 4. Cases de fuite autour du roi (cases accessibles non attaquées)
    escape_squares = list(board.attacks(king_square))
    safe_escape = sum(
        1 for sq in escape_squares
        if not board.is_attacked_by(not color, sq) and
        (not board.piece_at(sq) or board.piece_at(sq).color != color)
    )
    score += (9-safe_escape) * 3

    # 5. Pièces ennemies proches du roi
    danger_zone = 0
    for sq in chess.SQUARES:
        if chess.square_distance(sq, king_square) <= 2:
            piece = board.piece_at(sq)
            if piece and piece.color != color:
                danger_zone += 1
    score += danger_zone * 4
    return 1/score


######################################################################################################
##3 Évaluation des fous 

def evaluate_bishops(board, color):
    score = 0
    bishops = list(board.pieces(chess.BISHOP, color))
    
    # Bonus pour la paire de fous
    if len(bishops) >= 2:
        score += 15  # bonus de 0.3 point

    for sq in bishops:
        attacks = board.attacks(sq)
        score += len(attacks)  # activité
        if square_color(sq):
            couleur_du_fou = chess.BLACK
        couleur_du_fou = chess.WHITE
        # Bonus si le fou est centralisé
        if sq in [chess.D4, chess.D5, chess.E4, chess.E5]:
            score += 10
        
        # Bonus si le fou est sur une longue diagonale
        if chess.square_name(sq) in ['a1', 'h8', 'a8', 'h1', 'c1', 'f1', 'c8', 'f8']:
            score += 5
        #Pions bloquants les fou
        pawn_squares_alliés = board.pieces(chess.PAWN, couleur_du_fou)
        blocking_pawns = [sq for sq in pawn_squares_alliés if sq in attacks]
        score -= 5*len(blocking_pawns)
        #potentielles cases d'amélioration pour le fou
        attacked = board.attacks(sq)
        for piece_sq in attacked:
            attackers = 0
            defenders = 0
            for k in board.attackers(color, piece_sq):
                if k != sq:
                    piece = board.piece_at(k)
                    if piece and piece.piece_type != chess.PAWN:
                        defenders += 1
            for k in board.attackers(not color, piece_sq):
                piece = board.piece_at(k)
                if piece.piece_type != chess.PAWN:
                    attackers += 1
            if defenders > attackers and not is_attacked_by_pawn(piece_sq, board, not color):
                score += 4

    return score

######################################################################################################
##4 Évaluation de la dame

def evaluate_queen_activity(board, color):  # Plus c’est élevé, plus la dame est active
    score = 0
    queen_squares = board.pieces(chess.QUEEN, color)
    
    if not queen_squares:
        return -100  # Dame absente (échangée ou mat)

    for square in queen_squares:
        rank = chess.square_rank(square)
        file = chess.square_file(square)

        # 1. Centralisation
        if file in [3, 4] and rank in [3, 4]:
            score += 10  # Centre absolu
        elif 2 <= file <= 5 and 2 <= rank <= 5:
            score += 6   # Zone centrale
        else:
            score += 2   # Périphérie

        # 2. Mobilité (nombre de cases atteignables)
        mobility = len(board.attacks(square))
        score += mobility * 0.5  # pondéré légèrement

        # 3. Pression sur le camp adverse
        attack_targets = board.attacks(square)
        for target in attack_targets:
            piece = board.piece_at(target)
            if piece and piece.color != color:
                value = {
                    chess.PAWN: 1,
                    chess.KNIGHT: 3,
                    chess.BISHOP: 3,
                    chess.ROOK: 5,
                    chess.QUEEN: 9,
                    chess.KING: 10  # présence = menace, pas une vraie valeur
                }.get(piece.piece_type, 0)
                score += value  # plus elle menace, mieux c’est

        # 4. Est-elle attaquée ?
        if board.is_attacked_by(not color, square):
            score -= 12  # Menacée => danger potentiel

        # 5. Derrière un pion ou coincée ?
        blockers = list(board.attackers(color, square))
        if not blockers:
            score += 3  # libre
        else:
            score -= 2 * len(blockers)  # coincée par nos propres pièces
        attacked = board.attacks(square)
        for piece_sq in attacked:
            attackers = 0
            defenders = 0
            for k in board.attackers(color, piece_sq):
                if k != square:
                    piece = board.piece_at(k)
                    if piece and piece.piece_type != chess.PAWN:
                        defenders += 1
            for k in board.attackers(not color, piece_sq):
                piece = board.piece_at(k)
                if piece.piece_type != chess.PAWN:
                    attackers += 1
            if defenders > attackers and not is_attacked_by_pawn(piece_sq, board, not color):
                score += 4

    return score

######################################################################################################
##5 Évaluation des tours

def eval_efficacite_tours(board, color):
    score = 0
    rooks = board.pieces(chess.ROOK, color)

    for rook_square in rooks:
        # 1. Colonne ouverte ou semi-ouverte
        file = chess.square_file(rook_square)
        col = [chess.square(file, r) for r in range(8)]
        pions_sur_colonne = [board.piece_at(sq) for sq in col if board.piece_at(sq) and board.piece_at(sq).piece_type == chess.PAWN]

        if not pions_sur_colonne:
            score += 40  # colonne totalement ouverte
        elif all(p.color != color for p in pions_sur_colonne):
            score += 20  # semi-ouverte

        # 2. Tour connectée (protégée par une autre tour de la même couleur)
        attackers = board.attackers(color, rook_square)
        if any(board.piece_at(a).piece_type == chess.ROOK for a in attackers if board.piece_at(a) and board.piece_at(a).color == color):
            score += 15

        # 3. Sur la 7e rangée (rangée 6 si blancs, 1 si noirs)
        rank = chess.square_rank(rook_square)
        if (color == chess.WHITE and rank == 6) or (color == chess.BLACK and rank == 1):
            score += 25

    return score





In [8]:
import pandas as pd
#Création de la dataframe à partir de nos données FEN

colonnes = [
    "Sécurité_du_roi",
    "activité_reine",
    "activité_fous",
    "activité_tours",
    "activité_cavaliers",
    "différence_pièces",
    "évaluation"

]
def extraire_features(board, state, color, eval):
    return {
        "Sécurité_du_roi": evaluate_king_safety(board, chess.WHITE) - evaluate_king_safety(board, chess.BLACK),
        "activité_reine": evaluate_queen_activity(board, chess.WHITE) - evaluate_queen_activity(board, chess.BLACK),
        "activité_fous": evaluate_bishops(board, chess.WHITE) - evaluate_bishops(board, chess.BLACK),
        "activité_tours": eval_efficacite_tours(board, chess.WHITE) - eval_efficacite_tours(board, chess.BLACK),
        "activité_cavaliers": evaluate_knights(board, chess.WHITE) - evaluate_knights(board, chess.BLACK),
        "différence_pièces": eval_finale_pièces(board, state, color),
        "evaluation": eval
    }

# Création d'une DataFrame vide
df = pd.DataFrame(columns=colonnes)
positions_fen = []
traits = []
evaluations = []

with open("aggressive_games_1400.txt", "r") as f:
    for _, line in zip(range(1000), f):
        ligne = line.strip()
        if ";" not in ligne:
            continue  # ignore si mal formatée
        fen_str, eval_str = ligne.split(";")
        positions_fen.append(fen_str)
        trait = fen_str.split(" ")[1]
        traits.append(trait)
        evaluations.append(float(eval_str))
Colors = [chess.WHITE if t == 'w' else chess.BLACK for t in traits]
for fen, color, eval_ in zip(positions_fen, Colors, evaluations):
    board = chess.Board(fen)
    features = extraire_features(board, "mg", color, eval_)
    df.loc[len(df)] = list(features.values())

df


,Sécurité_du_roi,activité_reine,activité_fous,activité_tours,activité_cavaliers,différence_pièces,évaluation
0,-0.006194,5.5,-11.0,0.0,7.0,0.0,0.18
1,0.000000,0.0,-15.0,0.0,0.0,0.0,0.21
2,0.000000,-1.0,-15.0,0.0,14.0,0.0,0.13
3,0.000000,-3.0,-19.0,0.0,-17.0,0.0,0.21
4,-0.003884,2.0,-6.0,0.0,-14.0,0.0,0.13
...,...,...,...,...,...,...,...
995,-0.011622,152.5,-22.0,-30.0,-72.0,570.0,2.77
996,-0.011622,126.0,-22.0,-30.0,-65.0,-250.0,0.52
997,-0.011622,126.0,-14.0,-30.0,-61.0,250.0,1.68
998,-0.011622,121.0,-14.0,-30.0,-61.0,-250.0,0.17


In [ ]:

game = chess.pgn.read_game("game_data_2013_01.pgn")

AttributeError: 'str' object has no attribute 'readline'